데이터 전처리

In [3]:
import pandas as pd

# 엑셀 파일 불러오기
file_path = "C:\\Users\\정하민\\Downloads\\선용품 표준코드 목록 - ALL.xlsx"
df = pd.read_excel(file_path)

# 학습 및 전처리를 위한 기본 구조 정의
class SimplePreprocessor:
    def __init__(self, df: pd.DataFrame):
        self.df = df
        self.unit_dict = self._learn_units()
        self.synonyms = self._learn_synonyms()
        self.tfidf = self._learn_tfidf()
    
    def _learn_units(self):
        """데이터에서 단위 패턴 추출"""
        import re
        from collections import defaultdict

        units = defaultdict(int)
        specs = self.df['L5 NAME (SPEC)'].dropna().astype(str)
        for spec in specs:
            for match in re.findall(r'(\d+)\s*([A-Z]{1,5})', spec):
                units[match[1]] += 1
        return dict(units)

    def _learn_synonyms(self):
        """슬래시 기반 동의어 추출"""
        synonyms = {}
        for col in ['L1 NAME', 'L2 NAME', 'L3 NAME', 'L4 NAME']:
            if col in self.df.columns:
                for val in self.df[col].dropna().unique():
                    if '/' in val:
                        terms = [t.strip() for t in val.split('/')]
                        for t in terms:
                            synonyms[t] = terms
        return synonyms

    def _learn_tfidf(self):
        """TF-IDF 기반 단어 중요도 학습"""
        from sklearn.feature_extraction.text import TfidfVectorizer

        texts = self.df[['L1 NAME', 'L2 NAME', 'L3 NAME', 'L4 NAME']].fillna('').agg(' '.join, axis=1)
        vectorizer = TfidfVectorizer()
        tfidf_matrix = vectorizer.fit_transform(texts)
        tfidf_scores = dict(zip(vectorizer.get_feature_names_out(), tfidf_matrix.sum(axis=0).A1))
        return tfidf_scores

    def normalize_query(self, query: str):
        import re
        query = query.upper()
        for unit in self.unit_dict:
            query = re.sub(rf'(\d+)\s*{unit}', rf'\1{unit}', query)
        return query

    def expand_query(self, query: str):
        words = query.split()
        expanded = set(words)
        for word in words:
            if word in self.synonyms:
                expanded.update(self.synonyms[word])
        return ' '.join(expanded)

    def score_query(self, query: str):
        words = query.split()
        score = sum(self.tfidf.get(w.lower(), 0) for w in words)
        return score

    def preprocess(self, query: str):
        norm = self.normalize_query(query)
        expanded = self.expand_query(norm)
        score = self.score_query(expanded)
        return {
            'original': query,
            'normalized': norm,
            'expanded': expanded,
            'tfidf_score': score
        }

# 전처리기 인스턴스 생성
preprocessor = SimplePreprocessor(df)

# 테스트 쿼리
test_query = "3 Boxes BTL Stand"
result = preprocessor.preprocess(test_query)

# 결과 출력
import json
json.dumps(result, indent=2)


'{\n  "original": "3 Boxes BTL Stand",\n  "normalized": "3BOXES BTL STAND",\n  "expanded": "3BOXES STAND BTL",\n  "tfidf_score": 21.25872027176365\n}'

모델 학습

In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import re
from collections import defaultdict

# 1. 데이터 불러오기
file_path = "C:\\Users\\정하민\\Downloads\\선용품 표준코드 목록 - ALL.xlsx"
df = pd.read_excel(file_path, sheet_name='ALL')

# 2. 전처리기 정의
class SimplePreprocessor:
    def __init__(self, df: pd.DataFrame):
        self.df = df
        self.unit_dict = self._learn_units()
        self.synonyms = self._learn_synonyms()

    def _learn_units(self):
        units = defaultdict(int)
        specs = self.df['L5 NAME (SPEC)'].dropna().astype(str)
        for spec in specs:
            for match in re.findall(r'(\d+)\s*([A-Z]{1,5})', spec):
                units[match[1]] += 1
        return dict(units)

    def _learn_synonyms(self):
        synonyms = {}
        for col in ['L1 NAME', 'L2 NAME', 'L3 NAME', 'L4 NAME']:
            if col in self.df.columns:
                for val in self.df[col].dropna().unique():
                    if '/' in val:
                        terms = [t.strip() for t in val.split('/')]
                        for t in terms:
                            synonyms[t] = terms
        return synonyms

    def normalize_query(self, query: str):
        query = query.upper()
        for unit in self.unit_dict:
            query = re.sub(rf'(\d+)\s*{unit}', rf'\1{unit}', query)
        return query

    def expand_query(self, query: str):
        words = query.split()
        expanded = set(words)
        for word in words:
            if word in self.synonyms:
                expanded.update(self.synonyms[word])
        return ' '.join(expanded)

    def preprocess(self, query: str):
        norm = self.normalize_query(query)
        expanded = self.expand_query(norm)
        return expanded

# 3. 전처리기 인스턴스 생성
preprocessor = SimplePreprocessor(df)

# 4. 필수 컬럼 제거
required_cols = ['L1 NAME', 'L2 NAME', 'L3 NAME', 'L4 NAME', 'L5 NAME (SPEC)']
df = df.dropna(subset=required_cols)

# 5. 입력 텍스트 생성 및 전처리 적용
df["ITEM_NAME"] = (
    df["L3 NAME"].astype(str) + " " +
    df["L4 NAME"].astype(str) + " " +
    df["L5 NAME (SPEC)"].astype(str)
).str.upper()

df["ITEM_NAME"] = df["ITEM_NAME"].apply(preprocessor.preprocess)

# 6. 중복 제거
df_model = df[['ITEM_NAME', 'P CODE'] + required_cols].drop_duplicates()

# 7. 출력값 라벨 인코딩
label_encoders = {}
y_encoded = {}
for col in required_cols:
    le = LabelEncoder()
    y_encoded[col] = le.fit_transform(df_model[col].astype(str))
    label_encoders[col] = le

y_df = pd.DataFrame(y_encoded)

# 8. 학습/검증 분할
X_train, X_test, y_train_df, y_test_df = train_test_split(
    df_model["ITEM_NAME"],
    y_df,
    test_size=0.2,
    random_state=42
)

# 9. TF-IDF 벡터화
vectorizer = TfidfVectorizer(
    ngram_range=(1, 3),
    max_features=3000,
    min_df=2,
    max_df=0.8,
    stop_words='english'
)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# 10. 모델 학습 및 평가
models = {}
for col in required_cols:
    model = LogisticRegression(max_iter=300)
    model.fit(X_train_tfidf, y_train_df[col])
    y_pred = model.predict(X_test_tfidf)

    print(f"\n===== {col} 예측 성능 =====")
    print(classification_report(y_test_df[col], y_pred, zero_division=0))

    models[col] = model

# 11. 예측 함수 정의 (전처리기 포함)
def predict_item_name(raw_name):
    raw_name = raw_name.upper()
    preprocessed = preprocessor.preprocess(raw_name)
    vec = vectorizer.transform([preprocessed])
    prediction = {}
    for col in required_cols:
        label_id = models[col].predict(vec)[0]
        prediction[col] = label_encoders[col].inverse_transform([label_id])[0]
    return prediction

# ✅ 예측 테스트
print("\n예측 결과:")
print(predict_item_name("3 Boxes BTL Stand"))


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_47564\2271395991.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["ITEM_NAME"] = (
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_47564\2271395991.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["ITEM_NAME"] = df["ITEM_NAME"].apply(preprocessor.preprocess)



===== L1 NAME 예측 성능 =====
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       0.94      0.91      0.92        85
           2       1.00      0.29      0.44         7
           3       0.76      0.70      0.73       106
           4       0.87      0.68      0.76        38
           5       0.97      0.98      0.97       204
           6       0.99      0.99      0.99       441
           7       0.94      0.95      0.95       526
           8       0.80      0.68      0.73        96
           9       0.88      0.90      0.89       417
          10       0.92      0.97      0.94       896
          11       1.00      0.74      0.85        47
          12       0.96      1.00      0.98       229
          13       1.00      0.57      0.73        14
          14       0.73      0.98      0.84       203
          15       0.96      0.98      0.97        45
          16       0.85      0.52      0.64        33


c:\Users\정하민\Desktop\엠투코리아 모델\bert_cuda_env\Lib\site-packages\sklearn\metrics\_classification.py:98: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  type_true = type_of_target(y_true, input_name="y_true")
c:\Users\정하민\Desktop\엠투코리아 모델\bert_cuda_env\Lib\site-packages\sklearn\utils\multiclass.py:79: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  ys_types = set(type_of_target(x) for x in ys)
c:\Users\정하민\Desktop\엠투코리아 모델\bert_cuda_env\Lib\site-packages\sklearn\metrics\_classification.py:98: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  type_true = type_of_target(y_true, input_name="y_true")
c:\Users\정하민\Desktop\엠투코리아 모델\bert_cuda_env\Lib\site-packages\sklearn\utils\multiclass.py:79: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  ys_types = set(type_of_target(x) for x in ys)
c:\Users\정하민\Desktop\엠투코리아 모델\bert_cuda_env\Li


===== L5 NAME (SPEC) 예측 성능 =====
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         1
          10       0.00      0.00      0.00         1
          17       0.00      0.00      0.00         1
          20       0.00      0.00      0.00         1
          41       0.00      0.00      0.00         1
          42       0.00      0.00      0.00         1
          45       0.00      0.00      0.00         1
          68       0.00      0.00      0.00         1
          70       0.00      0.00      0.00         1
          71       0.00      0.00      0.00         1
          76       0.00      0.00      0.00         1
          80       0.00      0.00      0.00         1
          83       0.00      0.00      0.00         1
          85       0.00      0.00      0.00         1
          87       0.00      0.00      0.00         1
          88       0.00      0.00      0.00         1
          94       0.00      0.00      0.00    

c:\Users\정하민\Desktop\엠투코리아 모델\bert_cuda_env\Lib\site-packages\sklearn\metrics\_classification.py:98: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  type_true = type_of_target(y_true, input_name="y_true")
c:\Users\정하민\Desktop\엠투코리아 모델\bert_cuda_env\Lib\site-packages\sklearn\utils\multiclass.py:79: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  ys_types = set(type_of_target(x) for x in ys)
c:\Users\정하민\Desktop\엠투코리아 모델\bert_cuda_env\Lib\site-packages\sklearn\metrics\_classification.py:98: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  type_true = type_of_target(y_true, input_name="y_true")
c:\Users\정하민\Desktop\엠투코리아 모델\bert_cuda_env\Lib\site-packages\sklearn\utils\multiclass.py:79: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  ys_types = set(type_of_target(x) for x in ys)
c:\Users\정하민\Desktop\엠투코리아 모델\bert_cuda_env\Li

{'L1 NAME': 'GALLEY UTENSIL / TABLEWARE UTENSIL', 'L2 NAME': 'CUTLERY', 'L3 NAME': 'CUTLERY BOXES', 'L4 NAME': 'CUTLERY BENCH STAND PP', 'L5 NAME (SPEC)': '1/2"'}


모델 저장

In [5]:
# 모델 저장
import pickle

with open("logi_models.pkl", "wb") as f:
    pickle.dump({
        "models": models,
        "vectorizer": vectorizer,
        "label_encoders": label_encoders
    }, f)

print("\n✅ 모델 저장 완료: logi_models.pkl")


✅ 모델 저장 완료: logi_models.pkl


유사 품목, P CODE 추천

In [21]:
from sklearn.metrics.pairwise import cosine_similarity
from difflib import get_close_matches
import numpy as np
import pandas as pd

# 🔧 오타 보정 + 후보 선택 + 유사도 점수 표시
def autocorrect_input(user_input, top_n=3, cutoff=0.6):
    input_upper = user_input.upper()
    candidates = df_model["ITEM_NAME"].unique()

    # 1차 후보: 문자 기반 유사 품명
    close = get_close_matches(input_upper, candidates, n=top_n, cutoff=cutoff)
    if not close:
        return input_upper  # 후보 없음 → 원본 유지

    # TF-IDF 유사도 계산
    preprocessed_input = preprocessor.preprocess(input_upper)
    input_vec = vectorizer.transform([preprocessed_input])
    candidate_vecs = vectorizer.transform(close)
    similarities = cosine_similarity(input_vec, candidate_vecs).flatten()

    # 후보 리스트 정리
    scored_candidates = list(zip(close, similarities))
    scored_candidates.sort(key=lambda x: x[1], reverse=True)

    print("\n🔍 입력하신 품명과 유사한 후보를 찾았어요:")
    for i, (text, score) in enumerate(scored_candidates, 1):
        print(f"  {i}. {text}  (유사도: {round(score, 3)})")

    # 사용자 선택
    try:
        selection = int(input("👉 어떤 후보가 맞나요? (번호 입력, 0 = 원래 입력 유지): "))
        if selection in range(1, len(scored_candidates) + 1):
            return scored_candidates[selection - 1][0]
        else:
            return input_upper
    except ValueError:
        return input_upper


# 🔍 유사 품목 추천
def recommend_similar_items(input_name, top_n=5):
    preprocessed = preprocessor.preprocess(input_name.upper())
    input_vec = vectorizer.transform([preprocessed])
    all_vecs = vectorizer.transform(df_model["ITEM_NAME"])

    similarities = cosine_similarity(input_vec, all_vecs).flatten()
    top_indices = similarities.argsort()[::-1]

    recommendations = []
    for idx in top_indices:
        if similarities[idx] <= 0:
            continue

        row = df_model.iloc[idx]
        p_code = row["P CODE"] if "P CODE" in row and pd.notna(row["P CODE"]) else "N/A"

        rec = {
            "SIMILAR_ITEM_NAME": row["ITEM_NAME"],
            "SIMILARITY_SCORE": round(similarities[idx], 3),
            "L1 NAME": row["L1 NAME"],
            "L2 NAME": row["L2 NAME"],
            "L3 NAME": row["L3 NAME"],
            "L4 NAME": row["L4 NAME"],
            "L5 NAME (SPEC)": row["L5 NAME (SPEC)"],
            "P CODE": p_code
        }
        recommendations.append(rec)
        if len(recommendations) >= top_n:
            break

    return recommendations

def generate_structured_pcode_based_on_similar(similar_items, fallback_predicted):
    if not similar_items:
        return generate_structured_pcode(fallback_predicted)
    
    # 가장 유사한 기존 품목의 P CODE 접두부 (앞 8자리)
    closest_code = similar_items[0]['P CODE']
    prefix = closest_code[:8]

    # 중복 방지
    existing_codes = df["P CODE"].dropna().astype(str)
    used_suffixes = [
        int(code[-3:]) for code in existing_codes
        if code.startswith(prefix) and code[-3:].isdigit()
    ]

    next_suffix = max(used_suffixes, default=0) + 1
    return f"{prefix}{str(next_suffix).zfill(3)}"

def generate_structured_pcode(predicted):
    # 정확한 키 이름으로 추출
    l1 = predicted.get("L1 NAME", "")[:2].upper().ljust(2, "X")
    l2 = predicted.get("L2 NAME", "")[:2].upper().ljust(2, "X")
    l3 = predicted.get("L3 NAME", "")[:2].upper().ljust(2, "X")
    l4 = predicted.get("L4 NAME", "")[:2].upper().ljust(2, "X")

    prefix = f"{l1}{l2}{l3}{l4}"

    existing_codes = df["P CODE"].dropna().astype(str)
    used_suffixes = [
        int(code[-3:]) for code in existing_codes
        if code.startswith(prefix) and code[-3:].isdigit()
    ]

    next_suffix = max(used_suffixes, default=0) + 1
    new_pcode = f"{prefix}{str(next_suffix).zfill(3)}"

    return new_pcode

def recommend_similar_pcodes_detailed(new_pcode, top_n=5):
    existing = df.dropna(subset=["P CODE"])
    existing_codes = existing["P CODE"].astype(str).unique()

    # 문자 유사도 기반 후보 추출
    similar_codes = get_close_matches(new_pcode, existing_codes, n=top_n, cutoff=0.5)

    # 상세 정보 구성
    results = []
    for code in similar_codes:
        row = existing[existing["P CODE"] == code].iloc[0]
        results.append({
            "P CODE": code,
            "ITEM_NAME": row.get("ITEM_NAME") or row.get("L5 NAME (SPEC)", "N/A"),
            "L1": row.get("L1 NAME", ""),
            "L2": row.get("L2 NAME", ""),
            "L3": row.get("L3 NAME", ""),
            "L4": row.get("L4 NAME", ""),
            "L5": row.get("L5 NAME (SPEC)", "")
        })
    return pd.DataFrame(results)


# 🧠 전체 실행 함수
def interactive_product_recommendation():
    user_input = input("🔍 품명을 입력하세요: ").strip()
    if not user_input:
        print("❗ 입력이 비어 있습니다.")
        return

    # 1. 오타 보정
    corrected = autocorrect_input(user_input)
    if corrected != user_input.upper():
        print(f"🔧 오타 보정: '{user_input}' → '{corrected}'")
    else:
        print(f"✅ 입력 인식됨: '{corrected}'")

    # 2. 예측
    predicted = predict_item_name(corrected)

    # 3. 예측 신뢰도 평가
    def is_prediction_reasonable(predicted, user_input, threshold=0.3):
        pred_string = f"{predicted.get('L1 NAME', '')} {predicted.get('L2 NAME', '')} {predicted.get('L3 NAME', '')} {predicted.get('L4 NAME', '')} {predicted.get('L5 NAME (SPEC)', '')}"
        user_vec = vectorizer.transform([user_input.upper()])
        pred_vec = vectorizer.transform([pred_string.upper()])
        sim = cosine_similarity(user_vec, pred_vec)[0][0]
        print(f"📊 예측 신뢰도 (입력 vs 예측): {round(sim * 100, 1)}%")
        return sim >= threshold

    # 4. 유사 품목 추출
    similar_items = recommend_similar_items(corrected)

    # 5. 예측이 이상하면 fallback
    if not is_prediction_reasonable(predicted, corrected):
        print("⚠️ 예측된 분류가 입력과 너무 다릅니다. 유사 품목 기반으로 보정 중입니다.")
        if similar_items:
            fallback_input = similar_items[0]['SIMILAR_ITEM_NAME']
            predicted = predict_item_name(fallback_input)
            print(f"🔁 fallback 예측된 품명: '{fallback_input}' → 재예측됨")

    # 6. 분류 결과 출력
    print("\n🎯 예측된 분류:")
    for k, v in predicted.items():
        print(f"- {k}: {v}")

    # 7. 기존 P CODE 유무 확인
    matched = df[
        (df["L1 NAME"] == predicted["L1 NAME"]) &
        (df["L2 NAME"] == predicted["L2 NAME"]) &
        (df["L3 NAME"] == predicted["L3 NAME"]) &
        (df["L4 NAME"] == predicted["L4 NAME"]) &
        (df["L5 NAME (SPEC)"] == predicted["L5 NAME (SPEC)"])
    ]

    if not matched.empty:
        existing_pcode = matched["P CODE"].iloc[0]
        print(f"- 📦 추천 P CODE (기존): {existing_pcode}")
    else:
        # 8. 유사 품목 기반 신규 P CODE 생성
        new_pcode = generate_structured_pcode_based_on_similar(similar_items, predicted)
        print(f"- 🆕 추천 P CODE (신규): {new_pcode}")

        # 9. 유사 P CODE 및 품명 출력
        similar_details_df = recommend_similar_pcodes_detailed(new_pcode)
        if not similar_details_df.empty:
            print("\n📋 유사한 기존 P CODE 및 품명 (정렬된 테이블):")
            display(similar_details_df)
        else:
            print("📭 유사한 기존 P CODE 없음")



    # 유사 품목 추천
    print("\n🔎 유사한 품목 Top 5:")
    recommendations = recommend_similar_items(corrected)
    for rec in recommendations:
        print(f"[{rec['SIMILARITY_SCORE']}] {rec['SIMILAR_ITEM_NAME']} → {rec['P CODE']}")


최종 실행

In [29]:
interactive_product_recommendation()

✅ 입력 인식됨: 'CONSTRUCTION JACKET'
📊 예측 신뢰도 (입력 vs 예측): 66.2%

🎯 예측된 분류:
- L1 NAME: CLOTHING
- L2 NAME: WORKING CLOTHES
- L3 NAME: JACKET & TROUSERS WORKING
- L4 NAME: LIFE JACKET WITH WHISTLE
- L5 NAME (SPEC): XXL
- 🆕 추천 P CODE (신규): CGWCU1J0004

📋 유사한 기존 P CODE 및 품명 (정렬된 테이블):


,P CODE,ITEM_NAME,L1,L2,L3,L4,L5
0,CGWCU1J003,SLEEVE JACKET UNDRESSED SIZE L YELLOW EAR MUF...,CLOTHING,WORKING CLOTHES,UNDRESSED UNIFORM,JACKET UNIFORM UNDRESSED KHAKI LONG SLEEVE,SIZE M
1,CGWCU1J002,SLEEVE JACKET LL UNDRESSED SIZE LONG UNIFORM K...,CLOTHING,WORKING CLOTHES,UNDRESSED UNIFORM,JACKET UNIFORM UNDRESSED KHAKI LONG SLEEVE,SIZE LL
2,CGWCU1J001,SLEEVE JACKET UNDRESSED SIZE LONG UNIFORM KHAKI L,CLOTHING,WORKING CLOTHES,UNDRESSED UNIFORM,JACKET UNIFORM UNDRESSED KHAKI LONG SLEEVE,SIZE L
3,CGWCW2J004,JACKET WINTER SIZE HOODED L YELLOW EAR MUFF L...,CLOTHING,WORKING CLOTHES,WINTER HOODED JACKETS & TROUSERS,JACKET HOODED WINTER,SIZE M
4,CGWCU1T003,SIZE UNDRESSED L YELLOW EAR MUFF LEFT TROUSER...,CLOTHING,WORKING CLOTHES,UNDRESSED UNIFORM,TROUSERS UNIFORM UNDRESSED KHAKI,SIZE M



🔎 유사한 품목 Top 5:
[0.587] SLEEVE JACKET UNDRESSED SIZE LONG UNIFORM KHAKI L → CGWCU1J001
[0.583] SLEEVE JACKET UNDRESSED SIZE SHORT UNIFORM KHAKI L → CGWCU1J201
[0.555] LIFE JACKET TOILET LIFEBUOY SOAP 100GRMX24'S → CMHAT1S202
[0.555] LIFE JACKET TOILET LIFEBUOY SOAP 100GRM → CMHAT1S201
[0.553] XXL JACKET KHAKI TROUSERS 3L & COTTON WORKING → CGWCJ2J501
